# 4.8 - Grouping Data and Aggregating Variables

## Contents

#### 1- Import the libraries
#### 2- Import the dataframe created in previous exercise
#### 3- Calculate the mean of the “order_number” column grouped by the “department_id” & analyze the results
#### 4- Create a loyalty flag for existing customers using the transform() and loc() functions
#### 5- Use the loyalty flag and check the basic statistics of the product prices for each loyalty category
#### 6- Create a spending flag for each user based on the average price across all their orders
#### 7- Create an order frequency flag that marks the regularity of a user’s ordering behavior
#### 8- Export the dataframe with new columns

In [5]:
# Import Libraries
import pandas as pd
import numpy as np
import os

In [6]:
# Assigning the folder path to a variable
path = r"C:\Users\Toshiba\09-10-2023 Instacart Basket Analysis"

In [7]:
# Importing the dataframe orders_products_merged with os library
ords_prods_merged = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_merged.pkl'))

In [8]:
## Calculating the mean of the “order_number” column grouped by the “department_id”

ords_prods_merged.groupby('department_id').agg({'order_number': ['mean']})

,order_number
,mean
department_id,
1,15.457838
2,17.277920
3,17.170395
4,17.811403
5,15.215751
6,16.439806
7,17.225802
8,15.340650


In [9]:
ords_prods_merged.head(5)

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,_merge,product_name,aisle_id,department_id,prices,exists
0,2539329,1,1,2,8,11.0,196,1,0,both,Soda,77,7,9.0,both
1,2398795,1,2,3,7,15.0,196,1,1,both,Soda,77,7,9.0,both
2,473747,1,3,3,12,21.0,196,1,1,both,Soda,77,7,9.0,both
3,2254736,1,4,4,7,29.0,196,1,1,both,Soda,77,7,9.0,both
4,431534,1,5,4,15,28.0,196,1,1,both,Soda,77,7,9.0,both


In [ ]:
## Exercise 4.8 - Number 3
## The Department_id of 21 has the highest mean and the department_id of 5 has the lowest mean. So the company should focus
## on more production for department_id = 21.

## For the subset , the department id's were totally different.

In [10]:
## Exercise 4.8 - Number 4

## AGGREGATING DATA WITH TRANSFORM 

# Three-step process to find the loyal customer

# 1) Split the data into groups based on the “user_id” column.
# 2) Apply the transform() function on the “order_number” column to generate the maximum orders for each user.
# 3) Create a new column, “max_order,” into which you’ll place the results of your aggregation.

ords_prods_merged['max_order'] = ords_prods_merged.groupby(['user_id'])['order_number'].transform(np.max)

In [11]:
## This command tells pandas not to assign any options regarding the maximum number of rows to display.
## You should now see all 100 rows

pd.options.display.max_rows = None

In [12]:
ords_prods_merged.head(5)

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,_merge,product_name,aisle_id,department_id,prices,exists,max_order
0,2539329,1,1,2,8,11.0,196,1,0,both,Soda,77,7,9.0,both,10
1,2398795,1,2,3,7,15.0,196,1,1,both,Soda,77,7,9.0,both,10
2,473747,1,3,3,12,21.0,196,1,1,both,Soda,77,7,9.0,both,10
3,2254736,1,4,4,7,29.0,196,1,1,both,Soda,77,7,9.0,both,10
4,431534,1,5,4,15,28.0,196,1,1,both,Soda,77,7,9.0,both,10


In [13]:
## Deriving Columns with loc() by using the max_order column created by transform function

ords_prods_merged.loc[ords_prods_merged['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'


In [14]:
ords_prods_merged.loc[(ords_prods_merged['max_order'] <= 40) & (ords_prods_merged['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'

In [15]:
ords_prods_merged.loc[ords_prods_merged['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

In [16]:
## To check the frequency of the 'loyalty_flag'
ords_prods_merged['loyalty_flag'].value_counts(dropna=False)

loyalty_flag
Regular customer    15876776
Loyal customer      10284093
New customer         6243990
Name: count, dtype: int64

In [17]:
# Exercise 4.8 - Number 5

ords_prods_merged.groupby('loyalty_flag').agg({'prices': ['mean','min','max']})


prices              
                       mean  min      max
loyalty_flag                             
Loyal customer    10.386336  1.0  99999.0
New customer      13.294670  1.0  99999.0
Regular customer  12.495717  1.0  99999.0

In [ ]:
## The loyal customers are those whose order numbers are more but by seeing the mean, min and max value, I can say that 
## the price range does not differ much. 
## The regular customers are ordering products with more value and hence the mean is greater than the loyal customers.
## The New customers have the highest mean value. It shows even though their order numbers are less but they are ordring 
## products with more value.

In [18]:
# Exercise 4.8 - Number 6

#The team now wants to target different types of spenders in their marketing campaigns. 
#
#This can be achieved by looking at the prices of the items people are buying. Create a spending flag for each user based 
#on the average price across all their orders using the following criteria:
#
#If the mean of the prices of products purchased by a user is lower than 10, then flag them as a “Low spender.”
#If the mean of the prices of products purchased by a user is higher than or equal to 10, then flag them as a “High spender.”


ords_prods_merged['mean_order'] = ords_prods_merged.groupby(['user_id'])['prices'].transform(np.mean)

In [19]:
ords_prods_merged.head(5)

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,_merge,product_name,aisle_id,department_id,prices,exists,max_order,loyalty_flag,mean_order
0,2539329,1,1,2,8,11.0,196,1,0,both,Soda,77,7,9.0,both,10,New customer,6.367797
1,2398795,1,2,3,7,15.0,196,1,1,both,Soda,77,7,9.0,both,10,New customer,6.367797
2,473747,1,3,3,12,21.0,196,1,1,both,Soda,77,7,9.0,both,10,New customer,6.367797
3,2254736,1,4,4,7,29.0,196,1,1,both,Soda,77,7,9.0,both,10,New customer,6.367797
4,431534,1,5,4,15,28.0,196,1,1,both,Soda,77,7,9.0,both,10,New customer,6.367797


In [20]:
## Deriving Columns with loc() by using the mean_order column created by transform function

ords_prods_merged.loc[ords_prods_merged['mean_order'] < 10, 'spending_flag'] = 'Low Spender'


In [21]:
ords_prods_merged.loc[(ords_prods_merged['mean_order'] >= 10), 'spending_flag'] = 'High Spender'

In [22]:
## To check the frequency of the 'spending_flag'
ords_prods_merged['spending_flag'].value_counts(dropna=False)

spending_flag
Low Spender     31770614
High Spender      634245
Name: count, dtype: int64

In [ ]:
# There are 634245 High Spender and 31770614 low Spender.

In [25]:
# Exercise 4.8 - Number 7 
#Create an order frequency flag that marks the regularity of a user’s ordering behavior according to the median in 
#the “days_since_prior_order” column. The criteria for the flag should be as follows:
#
#If the median of “days_since_prior_order” is higher than 20, then the customer should be labeled a “Non-frequent customer.”
#If the median is higher than 10 and lower than or equal to 20, then the customer should be labeled a “Regular customer.”
#If the median is lower than or equal to 10, then the customer should be labeled a “Frequent customer.”


# Creating a column 'median_order' with the median values of the 'days_since_prior_order' column grouped by user_id

ords_prods_merged['median_order'] = ords_prods_merged.groupby(['user_id'])['days_since_prior_order'].transform(np.median)




In [26]:
## Deriving Columns with loc() by using the max_order column created by transform function

ords_prods_merged.loc[ords_prods_merged['median_order'] > 20, 'customer_flag'] = 'Non-frequent customer'


In [27]:
ords_prods_merged.loc[(ords_prods_merged['median_order'] <= 20) & (ords_prods_merged['median_order'] > 10), 'customer_flag'] = 'Regular customer'

In [28]:
ords_prods_merged.loc[ords_prods_merged['median_order'] <= 10, 'customer_flag'] = 'Frequent customer'

In [30]:
ords_prods_merged.head(20)

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,_merge,...,aisle_id,department_id,prices,exists,max_order,loyalty_flag,mean_order,spending_flag,median_order,customer_flag
0,2539329,1,1,2,8,11.0,196,1,0,both,...,77,7,9.0,both,10,New customer,6.367797,Low Spender,20.0,Regular customer
1,2398795,1,2,3,7,15.0,196,1,1,both,...,77,7,9.0,both,10,New customer,6.367797,Low Spender,20.0,Regular customer
2,473747,1,3,3,12,21.0,196,1,1,both,...,77,7,9.0,both,10,New customer,6.367797,Low Spender,20.0,Regular customer
3,2254736,1,4,4,7,29.0,196,1,1,both,...,77,7,9.0,both,10,New customer,6.367797,Low Spender,20.0,Regular customer
4,431534,1,5,4,15,28.0,196,1,1,both,...,77,7,9.0,both,10,New customer,6.367797,Low Spender,20.0,Regular customer
5,3367565,1,6,2,7,19.0,196,1,1,both,...,77,7,9.0,both,10,New customer,6.367797,Low Spender,20.0,Regular customer
6,550135,1,7,1,9,20.0,196,1,1,both,...,77,7,9.0,both,10,New customer,6.367797,Low Spender,20.0,Regular customer
7,3108588,1,8,1,14,14.0,196,2,1,both,...,77,7,9.0,both,10,New customer,6.367797,Low Spender,20.0,Regular customer
8,2295261,1,9,1,16,0.0,196,4,1,both,...,77,7,9.0,both,10,New customer,6.367797,Low Spender,20.0,Regular customer
9,2550362,1,10,4,8,30.0,196,1,1,both,...,77,7,9.0,both,10,New customer,6.367797,Low Spender,20.0,Regular customer


In [31]:
## Exporting the dataframe as pickle file

ords_prods_merged.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'orders_products_with_flags.pkl'))